In [9]:
import geopandas as gpd
import folium
from IPython.display import display

In [21]:
shapefile_paths = [
    "data-raw/MSS_nodes/dsm2_nodes_newcs_extranodes.shp",
    "data-raw/fc2024.01_chan/FC2024.01_channels_centerlines.shp"
]

In [27]:
nodes = gpd.read_file(shapefile_paths[0])
nodes.head

<bound method NDFrame.head of                X          Y   id                        geometry
0    586524.5000  4214278.5  329      POINT (586524.5 4214278.5)
1    595501.8125  4215164.0  328      POINT (595501.812 4215164)
2    587823.0625  4224132.5  327    POINT (587823.062 4224132.5)
3    611764.3125  4234500.0  326      POINT (611764.312 4234500)
4    609429.0625  4235459.0  325      POINT (609429.062 4235459)
..           ...        ...  ...                             ...
473          NaN        NaN  811   POINT (640955.51 4185200.504)
474          NaN        NaN  812  POINT (640236.914 4185018.152)
475          NaN        NaN  814  POINT (638627.967 4184416.817)
476          NaN        NaN  813  POINT (639130.486 4183087.804)
477          NaN        NaN  815  POINT (631626.258 4183226.381)

[478 rows x 4 columns]>

In [25]:
channels = gpd.read_file(shapefile_paths[1])
channels.head

<bound method NDFrame.head of       id                                           geometry
0      1  LINESTRING (652291.736 4172448.043, 652209.821...
1      2  LINESTRING (652401.235 4174797.442, 652581.296...
2      3  LINESTRING (651808.247 4176015.422, 651766.413...
3      4  LINESTRING (650024.633 4176799.368, 649933.498...
4      5  LINESTRING (648990.218 4180561.514, 648949.832...
..   ...                                                ...
546  819  LINESTRING (640236.667 4185024.396, 639714.773...
547  820  LINESTRING (639153.103 4183088.611, 639153.56 ...
548  821  LINESTRING (638934.638 4184888.455, 638822.624...
549  822  LINESTRING (638615.055 4184423.33, 638562.21 4...
550  823  LINESTRING (631626.258 4183226.381, 631559.926...

[551 rows x 2 columns]>

In [29]:
# nodes to highlight, based on figure 

nodes_to_highlight = [112, 176, 69]
nodes_filter = nodes[nodes['id'].isin(nodes_to_highlight)]
nodes_filter.head

<bound method NDFrame.head of                X           Y   id                       geometry
134  632155.4375  4186851.25  176  POINT (632155.438 4186851.25)
207  634764.4375  4193850.50  112   POINT (634764.438 4193850.5)
287  628267.0000  4185764.00   69         POINT (628267 4185764)>

In [36]:


import geopandas as gpd
import folium
from IPython.display import display

def create_multi_layer_map(shapefile_paths, filtered_gdf=None):
    """
    Creates an interactive map with multiple shapefile layers and optional filtered points in red.
    
    Parameters:
    - shapefile_paths: List of file paths to the shapefiles.
    - filtered_gdf: A GeoDataFrame with filtered points to display in red (optional).
    """
    # Initialize a list to store valid GeoDataFrames and their centroids
    gdfs = []
    all_centroids = []
    
    for path in shapefile_paths:
        # Read shapefile
        gdf = gpd.read_file(path)
        
        # Ensure the CRS is EPSG:4326
        if gdf.crs != "EPSG:4326":
            gdf = gdf.to_crs("EPSG:4326")

        if filtered_gdf.crs != "EPSG:4326":
            filtered_gdf = filtered_gdf.to_crs("EPSG:4326")
        
        # Check if there are valid geometries
        if gdf.is_empty.any():
            print(f"Warning: Some geometries in {path} are empty and will be ignored.")
            gdf = gdf[~gdf.is_empty]  # Remove empty geometries
        
        # Compute the centroid in EPSG:4326 (no need for reprojection)
        centroid = gdf.geometry.centroid
        center = [centroid.y.mean(), centroid.x.mean()]
        
        # Append to the list
        gdfs.append((gdf, center))
        all_centroids.append(center)
    
    # Calculate map center based on all layers
    avg_lat = sum([c[0] for c in all_centroids]) / len(all_centroids)
    avg_lon = sum([c[1] for c in all_centroids]) / len(all_centroids)
    
    # Create a Folium map centered on the average centroid
    m = folium.Map(location=[avg_lat, avg_lon], zoom_start=10)
    
    # Add each shapefile as a separate layer to the map
    for idx, (gdf, _) in enumerate(gdfs):
        folium.GeoJson(
            gdf, 
            name=f"Layer {idx + 1}: {shapefile_paths[idx].split('/')[-1]}"
        ).add_to(m)
    
    # If filtered_gdf is provided, add the filtered points in red
    if filtered_gdf is not None:
        for _, row in filtered_gdf.iterrows():
            folium.Marker(
                location=[row['geometry'].y, row['geometry'].x], 
                popup=f"ID: {row['id']}", 
                icon=folium.Icon(color='red')
            ).add_to(m)
    
    # Add layer control
    folium.LayerControl().add_to(m)
    
    # Display the map in the notebook
    display(m)

    


In [37]:

create_multi_layer_map(shapefile_paths, nodes_filter)

/var/folders/45/p22_x33n4s1dbmnzjdjzfr_80000gn/T/ipykernel_95984/642960532.py:34: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroid = gdf.geometry.centroid
/var/folders/45/p22_x33n4s1dbmnzjdjzfr_80000gn/T/ipykernel_95984/642960532.py:34: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroid = gdf.geometry.centroid


UnboundLocalError: cannot access local variable 'row' where it is not associated with a value